# Router LLM Serving Agent 튜토리얼


Router LLM Serving Agent는 클라우드 API 모델과 로컬 모델 간의 라우팅을 자동화하는 시스템이다. 네트워크 상태, 비용, 개인정보 보호 수준에 따라 OpenAI의 클라우드 모델(gpt-4o-mini)과 Ollama의 로컬 모델 중 최적의 선택을 한다.

기본 작동 원리는 다음과 같다:
1. 질문의 특성을 분석한다 (민감도, 복잡도)
2. 시스템 상태를 확인한다 (네트워크 상태, 로컬 모델 가용성)
3. 분석 결과에 따라 클라우드 또는 로컬 모델을 선택한다
4. 선택된 모델로 질문에 답변한다

이 시스템은 비용 절감, 응답 속도 개선, 개인정보 보호를 동시에 달성할 수 있다.

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

True

In [4]:
from openai import OpenAI
import requests
import json

# OpenAI 클라이언트 초기화 (클라우드 모델)
openai_client = OpenAI()

# Ollama 로컬 서버 주소
OLLAMA_BASE_URL = "http://localhost:11434"

In [5]:
# 모델 설정
model_config = {
    "cloud": {
        "name": "gpt-4o-mini",
        "type": "cloud",
        "description": "OpenAI 클라우드 모델",
        "advantages": ["높은 성능", "안정적인 서비스", "복잡한 추론 가능"]
    },
    "local": {
        "name": "exaone3.5:7.8b",
        "type": "local",
        "description": "Ollama 로컬 모델",
        "advantages": ["비용 절감", "빠른 응답", "개인정보 보호"]
    }
}

In [6]:
def check_local_model_available():
    """로컬 Ollama 모델이 사용 가능한지 확인한다"""
    
    # Ollama 서버에 연결 시도
    response = requests.get(f"{OLLAMA_BASE_URL}/api/tags")
    
    # 상태 코드가 200이면 사용 가능
    available = response.status_code == 200
    
    return available

In [7]:
def analyze_query_sensitivity(query):
    """질문의 민감도를 분석한다"""
    
    prompt = f"""
    다음 질문의 민감도를 분석하라.
    
    질문: {query}
    
    민감도 기준:
    - low: 일반적인 정보, 공개된 지식
    - medium: 개인적인 의견, 약간 민감한 주제
    - high: 개인정보, 기밀 정보, 매우 민감한 주제
    
    민감도만 답변하라 (low, medium, high 중 하나).
    """
    
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    
    sensitivity = response.choices[0].message.content.strip().lower()
    return sensitivity

In [8]:
def decide_model(query):
    """질문 특성과 시스템 상태를 고려하여 사용할 모델을 결정한다"""
    
    # 1. 로컬 모델 사용 가능 여부 확인
    local_available = check_local_model_available()
    print(f"로컬 모델 사용 가능: {local_available}")
    
    # 2. 질문 민감도 분석
    sensitivity = analyze_query_sensitivity(query)
    print(f"질문 민감도: {sensitivity}")
    
    # 3. 라우팅 결정
    # 로컬 모델이 없으면 무조건 클라우드
    selected_type = "cloud"
    reason = "로컬 모델 사용 불가"
    
    # 로컬 모델이 있는 경우
    # 민감도가 높으면 로컬 모델 사용 (개인정보 보호)
    selected_type = "local"
    reason = "개인정보 보호를 위한 로컬 처리"
    
    # 민감도가 낮거나 중간이면 클라우드 모델 사용 (높은 성능)
    selected_type = "cloud"
    reason = "높은 성능이 필요한 일반 질문"
    
    return selected_type, reason

In [9]:
def query_cloud_model(query):
    """OpenAI 클라우드 모델에 질문한다"""
    
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": query}],
        temperature=0.7
    )
    
    answer = response.choices[0].message.content
    return answer

In [10]:
def query_local_model(query):
    """Ollama 로컬 모델에 질문한다"""
    
    # Ollama API 요청
    payload = {
        "model": "exaone3.5:7.8b",
        "prompt": query,
        "stream": False
    }
    
    response = requests.post(
        f"{OLLAMA_BASE_URL}/api/generate",
        json=payload
    )
    
    result = response.json()
    answer = result.get("response", "")
    
    return answer

In [11]:
def route_to_model(model_type, query):
    """선택된 모델 타입에 따라 적절한 모델로 라우팅한다"""
    
    # 클라우드 또는 로컬 모델 선택
    answer = query_cloud_model(query)
    
    answer = query_local_model(query)
    
    return answer

In [12]:
def llm_serving_agent(query):
    """Router LLM Serving Agent의 전체 파이프라인을 실행한다"""
    
    print(f"질문: {query}\n")
    
    # 1단계: 모델 선택 결정
    model_type, reason = decide_model(query)
    selected_model = model_config[model_type]
    
    print(f"\n선택된 모델 타입: {model_type}")
    print(f"모델 이름: {selected_model['name']}")
    print(f"선택 이유: {reason}")
    print(f"모델 장점: {', '.join(selected_model['advantages'])}\n")
    
    # 2단계: 선택된 모델로 질문 처리
    answer = route_to_model(model_type, query)
    
    print(f"답변:\n{answer}\n")
    
    # 결과 반환
    result = {
        "query": query,
        "model_type": model_type,
        "model_name": selected_model['name'],
        "reason": reason,
        "answer": answer
    }
    
    return result

In [13]:
# 테스트 실행
test_queries = [
    "Python에서 리스트와 튜플의 차이는?",
    "우리 회사의 매출 데이터를 분석해줘",
    "홍길동의 전화번호와 주소를 알려줄래?"
]

results = []

for query in test_queries:
    print("=" * 80)
    result = llm_serving_agent(query)
    results.append(result)
    print("=" * 80)
    print()

질문: Python에서 리스트와 튜플의 차이는?

로컬 모델 사용 가능: True
질문 민감도: low

선택된 모델 타입: cloud
모델 이름: gpt-4o-mini
선택 이유: 높은 성능이 필요한 일반 질문
모델 장점: 높은 성능, 안정적인 서비스, 복잡한 추론 가능

답변:
Python에서 리스트와 튜플은 모두 순서가 있는 데이터 컬렉션을 나타내지만, 몇 가지 중요한 차이점이 있습니다:

### 1. **변경 가능성 (Mutability)**
- **리스트 (List):**
  - **변경 가능 (Mutable):** 리스트의 요소를 추가, 삭제, 수정할 수 있습니다.
  - 예시:
    ```python
    my_list = [1, 2, 3]
    my_list[0] = 10  # 요소 수정 가능
    my_list.append(4)  # 요소 추가 가능
    del my_list[2]  # 요소 삭제 가능
    ```

- **튜플 (Tuple):**
  - **불변 (Immutable):** 튜플의 요소는 생성 후 수정, 삭제할 수 없습니다.
  - 예시:
    ```python
    my_tuple = (1, 2, 3)
    # my_tuple[0] = 10  # 오류 발생: TypeError: 'tuple' object does not support item assignment
    ```

### 2. **성능**
- **튜플:** 일반적으로 리스트보다 메모리 사용량이 적고, 성능이 빠를 수 있습니다 (특히 큰 데이터셋에서).
- **리스트:** 동적 크기 조정이 가능하여 유연성이 높습니다.

### 3. **사용 목적**
- **리스트:** 데이터가 변경될 가능성이 있는 상황에서 사용합니다 (예: 사용자 입력 처리, 데이터 임시 저장 등).
- **튜플:** 데이터가 변경되지 않는 상황에서 사용합니다 (예: 좌표, 레코드 저장 등). 불변성 덕분에 코드 안전성을 높일 수 있습니다.

### 요약 테이블
| 특성      

In [13]:
# 결과 통계
print("\n===== 모델 사용 통계 =====")
model_usage = {}

for result in results:
    model_type = result["model_type"]
    model_usage[model_type] = model_usage.get(model_type, 0) + 1

print(f"총 질문 수: {len(results)}건\n")
print("모델 타입별 사용 횟수:")
for model_type, count in model_usage.items():
    percentage = (count / len(results)) * 100
    print(f"  - {model_type}: {count}건 ({percentage:.1f}%)")


===== 모델 사용 통계 =====
총 질문 수: 3건

모델 타입별 사용 횟수:
  - cloud: 3건 (100.0%)
